# Imports

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import mixture
import nltk
from nltk.tokenize import word_tokenize
from uk_stemmer import UkStemmer
from string import punctuation
from typing import List
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.model_selection import train_test_split

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


# Helper functions

In [8]:
stemmer = UkStemmer()

incorrect_stopwords = ['рос']

with open('./data/stopwords_ua.txt') as f_stopwords:
    stopwords = [x.replace('\n', '') for x in f_stopwords.readlines()]
    stopwords = [sw for sw in stopwords if sw not in incorrect_stopwords]

num_re = re.compile(r'\d+')
words_re = re.compile(r'\w+')

def prepare_uk(text: str) -> List[str]:
    tokens = word_tokenize(text.lower())
    tokens = [stemmer.stem_word(t) for t in tokens]
    tokens = [t for t in tokens if t not in punctuation]
    tokens = [t for t in tokens if t not in stopwords]
    tokens = [t for t in tokens if num_re.match(t) is None]
    tokens = [t for t in tokens if words_re.match(t) is not None]

    return ' '.join(tokens)

# Data import

## Read

In [15]:
df = pd.read_csv('./data/data_set_4.csv.zip')

df = df.drop(columns=['Unnamed: 0'])

df

,Link,Text,Label
0,https://t.me/c/1376264484/328,"Просто слухайте цей діалог. Ні, це не нарізка ...",True
1,https://t.me/c/1376264484/338,️ Рубль став найнестабільнішою валютою у всьом...,True
2,https://t.me/c/1376264484/350,Перше звернення мера Мелітополя Івана Федорова...,True
3,https://t.me/c/1376264484/361,"Росія загрожує Боснії ""українським сценарієм"" ...",True
4,https://t.me/c/1376264484/366,"Енергоатом повідомив, що окупанти пошкодили ви...",True
...,...,...,...
10730,https://t.me/warfakes/8878,"Громадян Таджикистану, який убив 11 людей на п...",False
10731,https://t.me/warfakes/8903,: Російська туристична компанія помилково відп...,False
10732,https://t.me/warfakes/8907,Відеозвернення губернатора Херсонської області...,False
10733,https://t.me/warfakes/8909,"Росіяни на Новий рік залишаться без текіли, бу...",False


## Preprocess text

In [16]:
df['text_preprocessed'] = df['Text'].apply(prepare_uk)

df

,Link,Text,Label,text_preprocessed
0,https://t.me/c/1376264484/328,"Просто слухайте цей діалог. Ні, це не нарізка ...",True,прост слухайт діалог нарізк фільм тарантін
1,https://t.me/c/1376264484/338,️ Рубль став найнестабільнішою валютою у всьом...,True,рубл найнестабільніш валют всьом світ
2,https://t.me/c/1376264484/350,Перше звернення мера Мелітополя Івана Федорова...,True,перш зверненн мер мелітопол іван федор післ зв...
3,https://t.me/c/1376264484/361,"Росія загрожує Боснії ""українським сценарієм"" ...",True,рос загрож босні українськ сценарі можлив всту...
4,https://t.me/c/1376264484/366,"Енергоатом повідомив, що окупанти пошкодили ви...",True,енергоат повідом окупант пошкодил високовольтн...
...,...,...,...,...
10730,https://t.me/warfakes/8878,"Громадян Таджикистану, який убив 11 людей на п...",False,громадян таджикистан уб люд полігон білгород п...
10731,https://t.me/warfakes/8903,: Російська туристична компанія помилково відп...,False,російськ туристичн компан помилков відправля б...
10732,https://t.me/warfakes/8907,Відеозвернення губернатора Херсонської області...,False,відеозверненн губернатор херсонськ област воло...
10733,https://t.me/warfakes/8909,"Росіяни на Новий рік залишаться без текіли, бу...",False,росіян нов залиш текіл бурбон коньяк французьк...


## Save preprocessed

In [17]:
df.to_csv('./data/preprocessed_uk.csv', index=False)

## Load preprocessed

In [18]:
df = pd.read_csv('./data/preprocessed_uk.csv')

df

,Link,Text,Label,text_preprocessed
0,https://t.me/c/1376264484/328,"Просто слухайте цей діалог. Ні, це не нарізка ...",True,прост слухайт діалог нарізк фільм тарантін
1,https://t.me/c/1376264484/338,️ Рубль став найнестабільнішою валютою у всьом...,True,рубл найнестабільніш валют всьом світ
2,https://t.me/c/1376264484/350,Перше звернення мера Мелітополя Івана Федорова...,True,перш зверненн мер мелітопол іван федор післ зв...
3,https://t.me/c/1376264484/361,"Росія загрожує Боснії ""українським сценарієм"" ...",True,рос загрож босні українськ сценарі можлив всту...
4,https://t.me/c/1376264484/366,"Енергоатом повідомив, що окупанти пошкодили ви...",True,енергоат повідом окупант пошкодил високовольтн...
...,...,...,...,...
10730,https://t.me/warfakes/8878,"Громадян Таджикистану, який убив 11 людей на п...",False,громадян таджикистан уб люд полігон білгород п...
10731,https://t.me/warfakes/8903,: Російська туристична компанія помилково відп...,False,російськ туристичн компан помилков відправля б...
10732,https://t.me/warfakes/8907,Відеозвернення губернатора Херсонської області...,False,відеозверненн губернатор херсонськ област воло...
10733,https://t.me/warfakes/8909,"Росіяни на Новий рік залишаться без текіли, бу...",False,росіян нов залиш текіл бурбон коньяк французьк...


## Vectorize

In [11]:
vectorizer = TfidfVectorizer(tokenizer=word_tokenize, min_df=.001)

vectorized = vectorizer.fit_transform(df['text_preprocessed']).toarray()

vectorizer.get_feature_names_out()

/mnt/e/university/master/text_analysis-labs/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array(['anonymous', 'bayraktar', 'bloomberg', ..., 'італ', 'їж', 'їхн'],
      dtype=object)

In [12]:
vectorizer.get_feature_names_out().shape

(1744,)

# Clustering

## GMM

In [19]:
true_fake_model = mixture.GaussianMixture(2)

true_fake_model.fit(vectorized)

GaussianMixture(n_components=2)

### Save model

In [21]:
gmm_name = './models/gmm_uk'
np.save(gmm_name + '_weights', true_fake_model.weights_, allow_pickle=False)
np.save(gmm_name + '_means', true_fake_model.means_, allow_pickle=False)
np.save(gmm_name + '_covariances', true_fake_model.covariances_, allow_pickle=False)

### Load the model

In [22]:
means = np.load('./models/gmm_uk' + '_means.npy')
covar = np.load('./models/gmm_uk' + '_covariances.npy')
true_fake_model = mixture.GaussianMixture(n_components = len(means), covariance_type='full')
true_fake_model.precisions_cholesky_ = np.linalg.cholesky(np.linalg.inv(covar))
true_fake_model.weights_ = np.load('./models/gmm_uk' + '_weights.npy')
true_fake_model.means_ = means
true_fake_model.covariances_ = covar

### Predict and add col

In [23]:
gmm_out = true_fake_model.predict(vectorized)

df['gmm_out'] = gmm_out
df['gmm_out'] = df['gmm_out'].astype(bool)

df

,Link,Text,Label,text_preprocessed,gmm_out
0,https://t.me/c/1376264484/328,"Просто слухайте цей діалог. Ні, це не нарізка ...",True,прост слухайт діалог нарізк фільм тарантін,False
1,https://t.me/c/1376264484/338,️ Рубль став найнестабільнішою валютою у всьом...,True,рубл найнестабільніш валют всьом світ,True
2,https://t.me/c/1376264484/350,Перше звернення мера Мелітополя Івана Федорова...,True,перш зверненн мер мелітопол іван федор післ зв...,True
3,https://t.me/c/1376264484/361,"Росія загрожує Боснії ""українським сценарієм"" ...",True,рос загрож босні українськ сценарі можлив всту...,False
4,https://t.me/c/1376264484/366,"Енергоатом повідомив, що окупанти пошкодили ви...",True,енергоат повідом окупант пошкодил високовольтн...,True
...,...,...,...,...,...
10730,https://t.me/warfakes/8878,"Громадян Таджикистану, який убив 11 людей на п...",False,громадян таджикистан уб люд полігон білгород п...,True
10731,https://t.me/warfakes/8903,: Російська туристична компанія помилково відп...,False,російськ туристичн компан помилков відправля б...,True
10732,https://t.me/warfakes/8907,Відеозвернення губернатора Херсонської області...,False,відеозверненн губернатор херсонськ област воло...,True
10733,https://t.me/warfakes/8909,"Росіяни на Новий рік залишаться без текіли, бу...",False,росіян нов залиш текіл бурбон коньяк французьк...,True


In [25]:
len(df[df['Label'] == df['gmm_out']]) / len(df)

0.4418258034466698

## K-means

In [27]:
true_false_model_km = sklearn.cluster.KMeans(n_clusters = 2, init = 'k-means++')
kmeans_out = true_false_model_km.fit_predict(vectorized)

In [29]:
kmeans_out

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [30]:
df['km_out'] = kmeans_out
df['km_out'] = df['km_out'].astype(bool)

df

,Link,Text,Label,text_preprocessed,gmm_out,km_out
0,https://t.me/c/1376264484/328,"Просто слухайте цей діалог. Ні, це не нарізка ...",True,прост слухайт діалог нарізк фільм тарантін,False,False
1,https://t.me/c/1376264484/338,️ Рубль став найнестабільнішою валютою у всьом...,True,рубл найнестабільніш валют всьом світ,True,False
2,https://t.me/c/1376264484/350,Перше звернення мера Мелітополя Івана Федорова...,True,перш зверненн мер мелітопол іван федор післ зв...,True,False
3,https://t.me/c/1376264484/361,"Росія загрожує Боснії ""українським сценарієм"" ...",True,рос загрож босні українськ сценарі можлив всту...,False,False
4,https://t.me/c/1376264484/366,"Енергоатом повідомив, що окупанти пошкодили ви...",True,енергоат повідом окупант пошкодил високовольтн...,True,False
...,...,...,...,...,...,...
10730,https://t.me/warfakes/8878,"Громадян Таджикистану, який убив 11 людей на п...",False,громадян таджикистан уб люд полігон білгород п...,True,False
10731,https://t.me/warfakes/8903,: Російська туристична компанія помилково відп...,False,російськ туристичн компан помилков відправля б...,True,False
10732,https://t.me/warfakes/8907,Відеозвернення губернатора Херсонської області...,False,відеозверненн губернатор херсонськ област воло...,True,False
10733,https://t.me/warfakes/8909,"Росіяни на Новий рік залишаться без текіли, бу...",False,росіян нов залиш текіл бурбон коньяк французьк...,True,False


In [32]:
len(df[df['Label'] == df['km_out']]) / len(df)

0.47498835584536564

# Classification

## Train-test-split etc

In [36]:
df_train, df_test = train_test_split(df, train_size=.7, shuffle=True, random_state=42)

len(df_train), len(df_test)

(7514, 3221)

In [42]:
vectorizer_classification = TfidfVectorizer(tokenizer=word_tokenize, min_df=.001)

X_train = vectorizer_classification.fit_transform(df_train['text_preprocessed'])
y_train = df_train['Label'].to_numpy(dtype=float)

X_test = vectorizer_classification.transform(df_test['text_preprocessed'])
y_test = df_test['Label'].to_numpy(dtype=float)

/mnt/e/university/master/text_analysis-labs/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## Logistic regression

In [51]:
lr_model = sklearn.linear_model.LogisticRegression(solver='liblinear')
lr_model.fit(X_train, y_train)
lr_pred =  lr_model.predict(X_test)

In [58]:
sklearn.metrics.accuracy_score(y_test, lr_pred)

0.8810928283141881

## SVM

### Default kernel

In [64]:
svm_model1 = sklearn.svm.SVC()
svm_model1.fit(X_train, y_train)
svm1_pred = svm_model1.predict(X_test)

sklearn.metrics.accuracy_score(y_test, svm1_pred)

0.889164855634896

### Linear

In [65]:
svm_model2 = sklearn.svm.SVC(kernel='linear')
svm_model2.fit(X_train, y_train)
svm2_pred = svm_model1.predict(X_test)

sklearn.metrics.accuracy_score(y_test, svm2_pred)

0.889164855634896

### Degree

In [66]:
svm_model3 = sklearn.svm.SVC(kernel='poly')
svm_model3.fit(X_train, y_train)
svm3_pred = svm_model3.predict(X_test)

sklearn.metrics.accuracy_score(y_test, svm3_pred)

0.8134119838559454